In [1]:
from lib.indicators.rsi import RSI
from lib.indicators.mfi import MFI
from lib.indicators.macd import MACD
from lib.indicators.nvi import NVI
from lib.indicators.koncorde import KONCORDE
from lib.indicators.ema import EMA

indicators = [
    MACD(23,12,9),
    RSI(30, 70, 14),
    EMA(14),
    MFI(20, 80, 14),
    NVI(255),
    KONCORDE(15)
]

In [2]:
from lib.strategies.RLstrategy import RL

2024-03-23 22:59:56.444846: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-23 22:59:56.446922: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-23 22:59:56.476557: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 22:59:56.476586: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 22:59:56.477299: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Necesitamos tener cargado el model_info.csv y la carpeta model con el modelo de Keras en el Root de algo_trader

In [3]:
strategy = RL(indicators)

2.15.0


In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime
# Binance Client
try:
  from binance.client import Client as BinanceProvider
except ModuleNotFoundError:
  !pip install python-binance
  from binance.client import Client as BinanceProvider

# Define the ticker symbol for Bitcoin (BTC-USD)
ticker = "BTCUSDT"

# Download historical data using yfinance
api_key = "OF6SkzXI0EAcvmMWlkeUKl6YyxYIFU4pN0Bj19gaVYZcgaTt7OImXxEyvoPcDhmk"
secret_key = "tXay1BDYuSyigxvl27UQIBJbIHADaep8FT7HPO9Mb3vfmcyDkz4keEaHkpm7dcFe"
provider= BinanceProvider(api_key, secret_key, tld='us')

klines = provider.get_historical_klines(ticker,BinanceProvider.KLINE_INTERVAL_1MINUTE, "23 Mar, 2024")
bitcoin_data = pd.DataFrame(klines, columns = ["Open time", "Open", "High", "Low", "Close", "Volume", "Close time", "Quote asset volume"," Number of trades"," Taker buy base asset volume", "Taker buy quote asset volume", "Ignore"])
bitcoin_data['Date'] = bitcoin_data['Open time'].apply(lambda x : datetime.fromtimestamp(x / 1000).strftime('%Y-%m-%d %H:%M:%S'))

bitcoin_data['Close'] =  bitcoin_data['Close'].apply(lambda x : float(x))
bitcoin_data["High"] = bitcoin_data["High"].apply(lambda x: float(x))
bitcoin_data["Low"] = bitcoin_data["Low"].apply(lambda x: float(x))
bitcoin_data["Volume"] = bitcoin_data["Volume"].apply(lambda x: float(x))
bitcoin_data["Open"] = bitcoin_data["Open"].apply(lambda x: float(x))

bitcoin_data = bitcoin_data.drop('Open time', axis=1)
bitcoin_data = bitcoin_data.set_index("Date")

# Display the first few rows of the data
print(bitcoin_data.head())
bitcoin_data.info()


                         Open      High       Low     Close   Volume  \
Date                                                                   
2024-03-22 21:00:00  63811.05  63834.63  63716.12  63816.47  0.19647   
2024-03-22 21:01:00  63670.83  63816.42  63670.83  63781.04  0.01215   
2024-03-22 21:02:00  63781.05  63781.07  63752.88  63752.89  0.08443   
2024-03-22 21:03:00  63655.96  63781.04  63655.96  63697.74  0.15912   
2024-03-22 21:04:00  63782.00  63810.98  63782.00  63810.98  0.06617   

                        Close time Quote asset volume   Number of trades  \
Date                                                                       
2024-03-22 21:00:00  1711152059999     12536.99390710                 21   
2024-03-22 21:01:00  1711152119999       773.67209910                  7   
2024-03-22 21:02:00  1711152179999      5384.95593900                 11   
2024-03-22 21:03:00  1711152239999     10135.66263490                  8   
2024-03-22 21:04:00  1711152299999     

In [5]:
import numpy as np

bitcoin_data_subset =  bitcoin_data[["Open", "High", "Low", "Close","Volume"]].copy()
bitcoin_data_subset["r"] = np.log(bitcoin_data_subset["Close"] / bitcoin_data_subset["Close"].shift(1))
bitcoin_data_subset = bitcoin_data_subset.rename_axis('Date')
bitcoin_data_subset.index = pd.to_datetime(bitcoin_data_subset.index)

bitcoin_data_subset.info()

df_except_last = bitcoin_data_subset.iloc[:-1]

# DataFrame con solo la última fila
df_last = bitcoin_data_subset.tail(1)
print(type(df_last))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1561 entries, 2024-03-22 21:00:00 to 2024-03-23 23:00:00
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1561 non-null   float64
 1   High    1561 non-null   float64
 2   Low     1561 non-null   float64
 3   Close   1561 non-null   float64
 4   Volume  1561 non-null   float64
 5   r       1560 non-null   float64
dtypes: float64(6)
memory usage: 85.4 KB
<class 'pandas.core.frame.DataFrame'>


In [6]:
strategy.prepare_data(df_except_last)

prepare antes del std                           Open      High       Low     Close   Volume  \
Date                                                                   
2024-03-22 21:00:00  63811.05  63834.63  63716.12  63816.47  0.19647   
2024-03-22 21:01:00  63670.83  63816.42  63670.83  63781.04  0.01215   
2024-03-22 21:02:00  63781.05  63781.07  63752.88  63752.89  0.08443   
2024-03-22 21:03:00  63655.96  63781.04  63655.96  63697.74  0.15912   
2024-03-22 21:04:00  63782.00  63810.98  63782.00  63810.98  0.06617   
...                       ...       ...       ...       ...      ...   
2024-03-23 22:55:00  64517.40  64517.40  64481.49  64481.49  0.00017   
2024-03-23 22:56:00  64481.49  64481.49  64481.49  64481.49  0.00000   
2024-03-23 22:57:00  64481.49  64481.49  64481.49  64481.49  0.00000   
2024-03-23 22:58:00  64481.50  64481.50  64481.50  64481.50  0.00081   
2024-03-23 22:59:00  64519.40  64519.40  64458.38  64458.38  0.00131   

                                r  MACD 

In [7]:
strategy.predict(df_last)

                             Open          High           Low         Close  \
Date                                                                          
2024-03-22 21:00:00   9939.699890   9942.252253   9925.872187   9940.396480   
2024-03-22 21:01:00   9917.855017   9939.415637   9918.815774   9934.876923   
2024-03-22 21:02:00   9935.026190   9933.909082   9931.599582   9930.491500   
2024-03-22 21:03:00   9915.538420   9933.904409   9916.498953   9921.899809   
2024-03-22 21:04:00   9935.174190   9938.568235   9936.136626   9939.541206   
...                           ...           ...           ...           ...   
2024-03-23 22:55:00  10049.742152  10048.608995  10045.120724  10043.998402   
2024-03-23 22:56:00  10044.147733  10043.015207  10045.120724  10043.998402   
2024-03-23 22:57:00  10044.147733  10043.015207  10045.120724  10043.998402   
2024-03-23 22:58:00  10044.149291  10043.016765  10045.122282  10043.999960   
2024-03-23 22:59:00  10050.053732  10048.920540  100

<Action.HOLD: 'hold'>